In [3]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
import numpy as np
from scipy.stats import chi2_contingency

sns.set_style("whitegrid")

from sklearn.compose import ColumnTransformer
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import BaggingClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix

### Carregar Dados


In [4]:
df_leads = pd.read_csv('.\datasets\leads.csv')

<>:1: SyntaxWarning: invalid escape sequence '\d'
<>:1: SyntaxWarning: invalid escape sequence '\d'
C:\Users\Carolina\AppData\Local\Temp\ipykernel_12840\3491388582.py:1: SyntaxWarning: invalid escape sequence '\d'
  df_leads = pd.read_csv('.\datasets\leads.csv')


In [5]:
df_leads.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9240 entries, 0 to 9239
Data columns (total 37 columns):
 #   Column                                         Non-Null Count  Dtype  
---  ------                                         --------------  -----  
 0   Prospect ID                                    9240 non-null   object 
 1   Lead Number                                    9240 non-null   int64  
 2   Lead Origin                                    9240 non-null   object 
 3   Lead Source                                    9204 non-null   object 
 4   Do Not Email                                   9240 non-null   object 
 5   Do Not Call                                    9240 non-null   object 
 6   Converted                                      9240 non-null   int64  
 7   TotalVisits                                    9103 non-null   float64
 8   Total Time Spent on Website                    9240 non-null   int64  
 9   Page Views Per Visit                           9103 

In [6]:
df_leads.head()

Prospect ID  Lead Number              Lead Origin  \
0  7927b2df-8bba-4d29-b9a2-b6e0beafe620       660737                      API   
1  2a272436-5132-4136-86fa-dcc88c88f482       660728                      API   
2  8cc8c611-a219-4f35-ad23-fdfd2656bd8a       660727  Landing Page Submission   
3  0cc2df48-7cf4-4e39-9de9-19797f9b38cc       660719  Landing Page Submission   
4  3256f628-e534-4826-9d63-4a8b88782852       660681  Landing Page Submission   

      Lead Source Do Not Email Do Not Call  Converted  TotalVisits  \
0      Olark Chat           No          No          0          0.0   
1  Organic Search           No          No          0          5.0   
2  Direct Traffic           No          No          1          2.0   
3  Direct Traffic           No          No          0          1.0   
4          Google           No          No          1          2.0   

   Total Time Spent on Website  Page Views Per Visit  ...  \
0                            0                   0.0  ...   
1                          674                   2.5  ...   
2                         1532                   2.0  ...   
3                          305                   1.0  ...   
4                         1428                   1.0  ...   

  Get updates on DM Content    Lead Profile    City  \
0                        No          Select  Select   
1                        No          Select  Select   
2                        No  Potential Lead  Mumbai   
3                        No          Select  Mumbai   
4                        No          Select  Mumbai   

  Asymmetrique Activity Index Asymmetrique Profile Index  \
0                   02.Medium                  02.Medium   
1                   02.Medium                  02.Medium   
2                   02.Medium                    01.High   
3                   02.Medium                    01.High   
4                   02.Medium                    01.High   

  Asymmetrique Activity Score Asymmetrique Profile Score  \
0                        15.0                       15.0   
1                        15.0                       15.0   
2                        14.0                       20.0   
3                        13.0                       17.0   
4                        15.0                       18.0   

  I agree to pay the amount through cheque  \
0                                       No   
1                                       No   
2                                       No   
3                                       No   
4                                       No   

  A free copy of Mastering The Interview Last Notable Activity  
0                                     No              Modified  
1                                     No          Email Opened  
2                                    Yes          Email Opened  
3                                     No              Modified  
4                                     No              Modified  

[5 rows x 37 columns]

### Feature Engineering and Data Cleaning

In [7]:
# Remover as colunas Prospect ID e Lead Number
df_leads.drop(columns=['Prospect ID', 'Lead Number'], axis=1, inplace=True) # Colunas com alta cardinalidade que não serve para treinamento do modelo preditivo

In [8]:
# Mostrar e Remover colunas categoricas que possuem somente um valor possível
for column in df_leads.select_dtypes(include=['object']).columns:
    # Usasse o select_dtypes(include=['object']).columns como iterator no dataset para encontrar somente as colunas do tipo categoricas
    if df_leads[column].nunique() == 1:
        # Usasse o nunique() para saber a quantidade de valores que existem em uma coluna, mas queremos trazer somente as colunas que possuem somente um valor
        print(f'A Coluna {column} possui somente um valor: {df_leads[column].unique()}')
        # Agora posso fazer o drop das colunas encontradas pelo iterator
        df_leads.drop(columns=[column], axis=1, inplace=True)

A Coluna Magazine possui somente um valor: ['No']
A Coluna Receive More Updates About Our Courses possui somente um valor: ['No']
A Coluna Update me on Supply Chain Content possui somente um valor: ['No']
A Coluna Get updates on DM Content possui somente um valor: ['No']
A Coluna I agree to pay the amount through cheque possui somente um valor: ['No']


In [9]:
# Mostrando  os valores possiveis para todas as colunas categoricas 
for column in df_leads.select_dtypes(include=['object']).columns:
    print(f'Coluna {column}: Valores = {df_leads[column].unique()}')

# Vamos verificas possiveis valores redundantes e sem utilidade dentro das colunas e corrigi-los
# Exemplo: A coluna Lead Source possui duas gráfias para o valor Google 

Coluna Lead Origin: Valores = ['API' 'Landing Page Submission' 'Lead Add Form' 'Lead Import'
 'Quick Add Form']
Coluna Lead Source: Valores = ['Olark Chat' 'Organic Search' 'Direct Traffic' 'Google' 'Referral Sites'
 'Welingak Website' 'Reference' 'google' 'Facebook' nan 'blog'
 'Pay per Click Ads' 'bing' 'Social Media' 'WeLearn' 'Click2call'
 'Live Chat' 'welearnblog_Home' 'youtubechannel' 'testone' 'Press_Release'
 'NC_EDM']
Coluna Do Not Email: Valores = ['No' 'Yes']
Coluna Do Not Call: Valores = ['No' 'Yes']
Coluna Last Activity: Valores = ['Page Visited on Website' 'Email Opened' 'Unreachable'
 'Converted to Lead' 'Olark Chat Conversation' 'Email Bounced'
 'Email Link Clicked' 'Form Submitted on Website' 'Unsubscribed'
 'Had a Phone Conversation' 'View in browser link Clicked' nan
 'Approached upfront' 'SMS Sent' 'Visited Booth in Tradeshow'
 'Resubscribed to emails' 'Email Received' 'Email Marked Spam']
Coluna Country: Valores = [nan 'India' 'Russia' 'Kuwait' 'Oman' 'United Arab 

In [10]:
# Mostrar o percentual de valores ausentes ou com valor igual a 'Select' para cada coluna no dataset
for column in df_leads.select_dtypes(include=['object']).columns:
    contagem = (df_leads[column]=='Select').sum() + df_leads[column].isnull().sum()
    print(f'{column}: {contagem / len(df_leads) * 100:.2f}%')

Lead Origin: 0.00%
Lead Source: 0.39%
Do Not Email: 0.00%
Do Not Call: 0.00%
Last Activity: 1.11%
Country: 26.63%
Specialization: 36.58%
How did you hear about X Education: 78.46%
What is your current occupation: 29.11%
What matters most to you in choosing a course: 29.32%
Search: 0.00%
Newspaper Article: 0.00%
X Education Forums: 0.00%
Newspaper: 0.00%
Digital Advertisement: 0.00%
Through Recommendations: 0.00%
Tags: 36.29%
Lead Quality: 51.59%
Lead Profile: 74.19%
City: 39.71%
Asymmetrique Activity Index: 45.65%
Asymmetrique Profile Index: 45.65%
A free copy of Mastering The Interview: 0.00%
Last Notable Activity: 0.00%


In [11]:
# Remover as colunas categoricas cujo percentuda de valores ausentes e valores iguais a 'Select' sejam maiores que 25%
for column in df_leads.select_dtypes(include=['object']).columns:
    contagem = (df_leads[column]=='Select').sum() + df_leads[column].isnull().sum()
    if (contagem / len(df_leads) *100) > 25:
        print(f'{column}: {contagem / len(df_leads) * 100:.2f}%')
        df_leads.drop(columns=[column], axis=1, inplace=True)

# Temos algumas formas de tratar esses dados, como substituir os valores ausentes pela MODA da coluna ou por valores de uma base de apoio
# Porem, para trabalhar com um dataset mais enxuto, foi usado uma abordagem mais radical e arbitraria quanto ao percentual de corte escolhido

Country: 26.63%
Specialization: 36.58%
How did you hear about X Education: 78.46%
What is your current occupation: 29.11%
What matters most to you in choosing a course: 29.32%
Tags: 36.29%
Lead Quality: 51.59%
Lead Profile: 74.19%
City: 39.71%
Asymmetrique Activity Index: 45.65%
Asymmetrique Profile Index: 45.65%


In [12]:
# Na coluna Lead Source, substituir o valor 'google' por 'Google'
df_leads['Lead Source'] = df_leads['Lead Source'].apply(lambda x: 'Google' if x == 'google' else x)

In [13]:
# Convertendo os valores das colunas categoricas de 'Yes/No' para 1/0
for column in df_leads.select_dtypes(include=['object']).columns:
    valores_unicos = df_leads[column].unique()
    if set(valores_unicos).issubset(set(['Yes', 'No'])):
        print(f'{column}')
        df_leads[column] = df_leads[column].apply(lambda x: 1 if x == 'Yes' else 0)

Do Not Email
Do Not Call
Search
Newspaper Article
X Education Forums
Newspaper
Digital Advertisement
Through Recommendations
A free copy of Mastering The Interview


In [14]:
# Removendo as linhas com valores ausentes das colunas categoricas 
colunas_categoricas = df_leads.select_dtypes(include=['object']).columns
df_leads.dropna(subset=colunas_categoricas, inplace=True)


In [15]:
# Apresentar Estatística Descritiva
df_leads.describe()

Do Not Email  Do Not Call    Converted  TotalVisits  \
count   9103.000000  9103.000000  9103.000000  9074.000000   
mean       0.079205     0.000220     0.379216     3.456028   
std        0.270073     0.014822     0.485219     4.858802   
min        0.000000     0.000000     0.000000     0.000000   
25%        0.000000     0.000000     0.000000     1.000000   
50%        0.000000     0.000000     0.000000     3.000000   
75%        0.000000     0.000000     1.000000     5.000000   
max        1.000000     1.000000     1.000000   251.000000   

       Total Time Spent on Website  Page Views Per Visit       Search  \
count                  9103.000000           9074.000000  9103.000000   
mean                    483.773921              2.370151     0.001538   
std                     545.519186              2.160871     0.039189   
min                       0.000000              0.000000     0.000000   
25%                      12.000000              1.000000     0.000000   
50%                     247.000000              2.000000     0.000000   
75%                     924.000000              3.200000     0.000000   
max                    2272.000000             55.000000     1.000000   

       Newspaper Article  X Education Forums    Newspaper  \
count        9103.000000         9103.000000  9103.000000   
mean            0.000220            0.000110     0.000110   
std             0.014822            0.010481     0.010481   
min             0.000000            0.000000     0.000000   
25%             0.000000            0.000000     0.000000   
50%             0.000000            0.000000     0.000000   
75%             0.000000            0.000000     0.000000   
max             1.000000            1.000000     1.000000   

       Digital Advertisement  Through Recommendations  \
count            9103.000000              9103.000000   
mean                0.000439                 0.000769   
std                 0.020959                 0.027721   
min                 0.000000                 0.000000   
25%                 0.000000                 0.000000   
50%                 0.000000                 0.000000   
75%                 0.000000                 0.000000   
max                 1.000000                 1.000000   

       Asymmetrique Activity Score  Asymmetrique Profile Score  \
count                  4944.000000                 4944.000000   
mean                     14.313511                   16.340210   
std                       1.394627                    1.807428   
min                       7.000000                   11.000000   
25%                      14.000000                   15.000000   
50%                      14.000000                   16.000000   
75%                      15.000000                   18.000000   
max                      18.000000                   20.000000   

       A free copy of Mastering The Interview  
count                             9103.000000  
mean                                 0.317258  
std                                  0.465434  
min                                  0.000000  
25%                                  0.000000  
50%                                  0.000000  
75%                                  1.000000  
max                                  1.000000

In [16]:
# Mostrar e remover colunas numéricas que possuem somente um valor possível
for column in df_leads.select_dtypes(include=['number']).columns:
    if df_leads[column].nunique() == 1:
        print(f'{column}')
        df_leads.drop(columns=column, axis=1, inplace=True)

In [17]:
# Mostrar o percentual de valores ausentes em cada coluna numerica
for column in df_leads.select_dtypes(include=['number']).columns:
    contagem = df_leads[column].isnull().sum()
    print(f'{column}: {contagem / len(df_leads) * 100:.2f}')

Do Not Email: 0.00
Do Not Call: 0.00
Converted: 0.00
TotalVisits: 0.32
Total Time Spent on Website: 0.00
Page Views Per Visit: 0.32
Search: 0.00
Newspaper Article: 0.00
X Education Forums: 0.00
Newspaper: 0.00
Digital Advertisement: 0.00
Through Recommendations: 0.00
Asymmetrique Activity Score: 45.69
Asymmetrique Profile Score: 45.69
A free copy of Mastering The Interview: 0.00


In [18]:
# Remover colunas numericas cujo percentual de valores ausentes seja maior que 25%
for column in df_leads.select_dtypes(include=['number']).columns:
    contagem_nulas = df_leads[column].isnull().sum()
    percentual_nulas = (contagem_nulas / len(df_leads) * 100)
    if percentual_nulas > 25:
        print(f'{column}: {percentual_nulas:.2f}')
        df_leads.drop(columns=[column], axis=1, inplace=True)

Asymmetrique Activity Score: 45.69
Asymmetrique Profile Score: 45.69


In [19]:
# Remover as linhas com valores ausentes das colunas numericas
colunas_numericas = df_leads.select_dtypes(include=['number']).columns
df_leads.dropna(subset=colunas_numericas, inplace=True)

### EDA
Hit Ratio
- Razão entre leads convertidos em vendas sobre quantidade total de leads (Conceito 1)
    100 leads criados e 30 foram convertidos = 30%
- Razão entre leads convertidos em vendas sobre a quantidade de leads encerrados (Conceito 2)
    100 leads criados, 20 leads que não foram convertidos e 20 leads que foram convertidos = 50%

In [20]:
# Distribuição da variável target em percentual
fig = px.bar(df_leads['Converted'].value_counts() / len(df_leads) * 100,
             title='Hit Ratio - Fator de Conversão',
             labels={'idex': 'Converted', 'value': 'Percentual'},
             opacity=0.8
             )
fig.update_layout(showlegend=False)
fig.show()

- Temos uma base de dados balanceada. Os percentuais de converted (target) motram que podemos usar uma abordagem de classificação sem antes precisar usar técnicas de detecção de anomalia para encontrar possíveis outliers.

In [21]:
# Matriz de Correlação das variáveis numéricas com o Plotly Go
corr_matrix = df_leads.select_dtypes(include=['number']).corr()

In [22]:
# Plot de Correlação
fig = go.Figure()

fig.add_trace(
    go.Heatmap(
        x = corr_matrix.columns,
        y = corr_matrix.index,
        z = np.array(corr_matrix),
        text = corr_matrix.values,
        texttemplate='%{text:.2f}',
        colorscale=px.colors.diverging.RdBu,
        zmin=-1,
        zmax=1
    )
)

fig.show()

- Observando a variável target na Matriz de Correlação, não encontramos uma Correlação com variáveis numéricas que chame atenção para usar no modelo de classificação. Percebemos que existem outras variáveis que tem uma correlação altissima entre elas, o que poderia ser criterio para um Data Cleaning

In [23]:
# Avaliando a divizão da distribuição correlacional entre os Quats da variável target e as númericas de maior percentual de correlação 

# BoxPlot Converted X TotalVisits
fig = px.box(df_leads, x='Converted', y='TotalVisits', color='Converted')

fig.show()

 - Em termos de mediana é possível perceber que não ha uma variação na distribuição da variavel Converted
 - Por mais que tenhamos um upperfance de 50% ha mais.
 
 ****A título de curiosidade, podemos fazer a remoção desses outliers

In [24]:
# BoxPlot Converted X TotalVisits
fig = px.box(df_leads, x='Converted', y='Total Time Spent on Website', color='Converted')

fig.show()

- Nessa correlação é percepitivel a difença entre a distribuição da variávael Target.
- As medianas do BloxPlot mostram uma diferença de mais de 800 pessoas entre o __conjunto de foi convertido e não foi convertido__


In [25]:
# BoxPlot Converted X TotalVisits
fig = px.box(df_leads, x='Converted', y='Page Views Per Visit', color='Converted')

fig.show()

- Observando pelo Gráfico de Matriz de Correlação podemos perceber que o correlação de algumas variáveis são quase 0. Mas devemos fazer uma analise minuciosa para conseguir mensular e estimar a influencia de cada variavel possivel.

#### Explorando Categóricas

In [26]:
# Criar uma tabela de contingencia de Converted x Lead Source
contingency_table_lead_source = pd.crosstab(df_leads['Converted'], df_leads['Lead Source'])

# Mostrar a tabela de contingencia
contingency_table_lead_source

Lead Source  Click2call  Direct Traffic  Facebook  Google  Live Chat  NC_EDM  \
Converted                                                                      
0                     1            1725        22    1726          0       0   
1                     3             818         9    1147          2       1   

Lead Source  Olark Chat  Organic Search  Pay per Click Ads  Press_Release  \
Converted                                                                   
0                  1305             718                  1              2   
1                   448             436                  0              0   

Lead Source  Reference  Referral Sites  Social Media  WeLearn  \
Converted                                                       
0                   33              94             1        0   
1                  410              31             1        1   

Lead Source  Welingak Website  bing  blog  testone  welearnblog_Home  \
Converted                                                              
0                           2     5     1        1                 1   
1                         127     1     0        0                 0   

Lead Source  youtubechannel  
Converted                    
0                         1  
1                         0

- Observando a Tabela podemos avaliar os fatos, que são.

    1 - Temos 62% de Not Converted X 38% Converted, isso deixa evidente que em alguns 
    casos temos um numero mais alto de pessoal que não foram convertidas.

    2 - Apesar do numero alto de Not Converted, em alguns casos temos uma troca de dominancia entre os valores da variável.

In [27]:
# Executar o teste de Independência de qui-quadrado
chi2, p_value, dof, expected = chi2_contingency(contingency_table_lead_source)

# Mostrando Teste de Hipotese
print(f'Estatística qui-quadrado: {chi2}')
print(f'P-Value: {p_value}')
print(f'Graus de Liberdade: {dof}') # Numero de valores em que a variável Target esta sendo distribuida -1
print(f'Existe uma relação entre Converted e Lead Source? {p_value < 0.05}')

Estatística qui-quadrado: 942.1372507753774
P-Value: 1.1748671316223743e-187
Graus de Liberdade: 19
Existe uma relação entre Converted e Lead Source? True


In [28]:
# Criar uma tabela de contingência de Converted x Lead Origin
contingency_table_lead_origin = pd.crosstab(df_leads['Converted'], df_leads['Lead Origin'])

# Mostrar Tabela de contingência
contingency_table_lead_origin

Lead Origin   API  Landing Page Submission  Lead Add Form  Lead Import
Converted                                                             
0            2463                     3118             37           21
1            1115                     1767            544            9

- Olhando a tabela conseguimos notar a distribuição sendo contundente aos valores percebidos anteriormente.
- Também notasse um diferença da proporção quando distribuida na origem __Lead Add Form__

In [29]:
# Executar teste de Independência de qui_quadrado
origin_chi2, origin_p, origin_dof, expected = chi2_contingency(contingency_table_lead_origin)

# Mostrar teste de Hipótese
print(f'Estatística qui-quadrado: {origin_chi2}')
print(f'P-Value: {origin_p}')
print(f'Graus de Liberdade: {origin_dof}')
print(f'Existe uma relação entre Converted e Lead Source? {p_value < 0.05}')


Estatística qui-quadrado: 843.1212236836468
P-Value: 1.9228780932726904e-182
Graus de Liberdade: 3
Existe uma relação entre Converted e Lead Source? True


### Preparação dos Dados

In [30]:
# Preparar os dados para o modelo
X = df_leads.drop(columns=['Converted'])
y = df_leads['Converted']

In [31]:
# Criar ColumnTransformer para Normalizar Numericas e One-Hot Encoding nas Categoricas
numeric_features = X.select_dtypes(include=['number']).columns
categorical_features = X.select_dtypes(include=['object']).columns
preprocessor = ColumnTransformer(transformers=[
    ('num', StandardScaler(), numeric_features),
    ('cat', OneHotEncoder(handle_unknown='ignore'), categorical_features) 
    # Com a divisão de conjunto entre treino e teste, podemos ter um valor num conjunto que não tem no outro. 
    # Para não ter um erro diagnosticado, usamos o handle_unknow='ignore' para não levar em concideração essa possível diferença 
])

In [32]:
# Dividir conjuntos
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=51)

# Aplicar ColumnTransformer
X_train_transformed = preprocessor.fit_transform(X_train)
X_test_transformed = preprocessor.transform(X_test)

# Mostrar Conjuntos
X_train.shape, X_test.shape

((7259, 16), (1815, 16))

### Trainamento do Modelo

In [33]:
# Criar o Modelo de BaggingClassifier

bagging_model = BaggingClassifier(
    estimator=LogisticRegression(), # Algoritmo de Classificação
    n_estimators=10, # Número arbitrario de samples para treinamento
    random_state=51

)

# Treinar o Modelo
bagging_model.fit(X_train_transformed, y_train)

BaggingClassifier(estimator=LogisticRegression(), random_state=51)

### Análise das Metricas (Avaliação do Modelo)
- Vamos comparar a predição com o valor real do modelo, conciderando o conjunto de testes

In [34]:
# Fazer predições no conjunto de testes
y_pred = bagging_model.predict(X_test_transformed)

In [35]:
# Avaliar Modelo
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)

# Mostrar Resultados
print(f'Acurácia: {accuracy}')
print(f'Precisão: {precision}')
print(f'Recall: {recall}')
print(f'F1 Score: {f1}')

Acurácia: 0.7972451790633609
Precisão: 0.7467320261437909
Recall: 0.682089552238806
F1 Score: 0.7129485179407177


In [36]:
# Mostrar uma Matriz de Confusão em Plotly
conf_matrix = confusion_matrix(y_test, y_pred)

fig = px.imshow(conf_matrix,
                labels=dict(x='Predição', y='Real', color='Contagem'),
                x=['Not Converted', 'Converted'],
                y=['Not Converted', 'Converted'],
                color_continuous_scale='Viridis')

fig.update_traces(text=conf_matrix, texttemplate='%{z}')
fig.update_layout(coloraxis_showscale=False)

fig.show()

In [37]:
# Extrair os coeficientes usados pelo algoritmo de Bagging e Calcular a inportancia das variáveis
# Com isso podemos descobrir uma variável estratégica que pode ser adotada pela empresa

importances = np.mean([np.abs(estimator.coef_[0]) for estimator in bagging_model.estimators_], axis=0)

# Tirar uma média da importancia das variáveis de todos os estimadores
# 1º Passo: Criar uma iteração nos estimadores do bagging_model (usando parametro do modelo)
# 2º Passo: Obter o coeficiente que da a importancia das variáveis, convertendo o coeficiente 
# em valor absoluto (usando numpy abs para transformar os coeficientes em valor absoluto e puxando o primeiro valor de coeficiente)
# 3º Passo: Jogar tudo dentro do Método np.mean() para tirar a média dos coeficiente no eixo das linhas

In [38]:
# Obter os nomes das features após o preprocessor
features_names = (numeric_features.tolist() +
                  preprocessor.named_transformers_['cat']
                  .get_feature_names_out(categorical_features).tolist())

# 1º Passo: Pegar todas as features numericas e transformar numa lista
# 2º Passo: Concatenar essa lista com as features categoricas criadas no preprocessor, 
# depois do OneHotEncoder
# 3º Passo: Pegar os nomes de saida que o Preprocessor criou a partir 
# das features categoricas originais transformada em lista

In [39]:
# Criar um DataFrame combinando os nomes das features e as importâncias
df_feature_importances = pd.DataFrame({'Feature': features_names, 'Importance': importances})

# Mostrar importancias em ordem decrescente
df_feature_importances = df_feature_importances.sort_values(by='Importance', ascending=False)

In [40]:
df_feature_importances

Feature  Importance
14               Lead Origin_Lead Add Form    2.382901
29            Lead Source_Welingak Website    1.710818
65       Last Notable Activity_Unreachable    1.392217
43  Last Activity_Had a Phone Conversation    1.202485
3              Total Time Spent on Website    1.176469
..                                     ...         ...
6                        Newspaper Article    0.064843
5                                   Search    0.064434
7                       X Education Forums    0.053349
11  A free copy of Mastering The Interview    0.035519
20                   Lead Source_Live Chat    0.029063

[68 rows x 2 columns]

In [41]:
# Plotar a Importancia das Features
fig = px.bar(df_feature_importances,
             x='Importance',
             y='Feature',
             orientation='h',
             title='Importancia das Features (Com base nos coeficientes absolutos)'
             )

fig.update_layout(height=1280, width=1000, yaxis={'categoryorder': 'total ascending'})

fig.show()

- Essas são as variáveis mais importantes para este modelo, o grau de importancia foi colocado em ordem decrescente para ficar mais claro quais as variáveis que o modelo levou em conta para calcular os coeficientes da Regreção Logistica em Bagging

### Some more Things

In [42]:
bagging_model.estimators_samples_

[array([6647, 2395, 3964, ..., 1386, 4404, 2518],
       shape=(7259,), dtype=int32),
 array([ 914, 6214, 4940, ..., 7166,  151, 1272],
       shape=(7259,), dtype=int32),
 array([6067, 5889, 4247, ..., 5644, 3350,  728],
       shape=(7259,), dtype=int32),
 array([ 405, 6461, 1538, ..., 6934, 1805, 2162],
       shape=(7259,), dtype=int32),
 array([3226, 4034, 4872, ..., 3994,  140, 4734],
       shape=(7259,), dtype=int32),
 array([ 514, 5133, 4920, ..., 6956, 7153, 1234],
       shape=(7259,), dtype=int32),
 array([1246, 4026, 3914, ..., 6846, 1043, 2326],
       shape=(7259,), dtype=int32),
 array([2697, 5933, 5075, ...,  543, 3580, 1019],
       shape=(7259,), dtype=int32),
 array([5528, 2923, 4766, ..., 5378, 5127,  672],
       shape=(7259,), dtype=int32),
 array([5310, 3010, 5471, ..., 4084, 4783, 4111],
       shape=(7259,), dtype=int32)]

- Pegou pra cada estimador LogisticRegression extriiu um sample dos dados de treinamento


In [43]:
bagging_model.estimators_samples_[0]

array([6647, 2395, 3964, ..., 1386, 4404, 2518],
      shape=(7259,), dtype=int32)

- Podemos observar que ele usou toda a base de dados de treinamento e cada um dos estimadores foi treinado com o conjunto, sorteando os valores em ordens diferentes

In [44]:
# Predizer a Probabilidade de Conversão
y_pred_prob = bagging_model.predict_proba(X_train_transformed)

In [45]:
y_pred_prob

array([[0.94462405, 0.05537595],
       [0.9280015 , 0.0719985 ],
       [0.91636572, 0.08363428],
       ...,
       [0.63791065, 0.36208935],
       [0.51214849, 0.48785151],
       [0.67731414, 0.32268586]], shape=(7259, 2))

### Cenário de CRM - Utilidade da Probabilidade

- CRM

    Leads Concluidos - Resultado Positivo ou Negativo

    Leads em Aberto - Não tenho Resultado

Treine um modelo no que está concluido, para que ele generalize bem no que está em aberto.

#### Decisões estratégicas, a depender da empresa
Leads em aberto
- Probabilidade de Converter
    Quando muito alto, podemos olhar com mais foco pra converter

    Quando muito Baixa, podemos pensar em descartar
    
    Probabilidade + Importância das Features

### Salvar dados e pre-processador do modelo

In [46]:
# Salvar DataFrame como CSV
df_leads.to_csv('.\leads_cleaned.csv', index=False) 
# usamos index=False para o pandas não criar mais uma coluna no DataFrame

<>:2: SyntaxWarning:

invalid escape sequence '\l'

<>:2: SyntaxWarning:

invalid escape sequence '\l'

C:\Users\Carolina\AppData\Local\Temp\ipykernel_12840\3142269350.py:2: SyntaxWarning:

invalid escape sequence '\l'



In [47]:
# Salvar o Preprocessor
import joblib

joblib.dump(preprocessor, '.\preprocessor_leads.pkl')

<>:4: SyntaxWarning:

invalid escape sequence '\p'

<>:4: SyntaxWarning:

invalid escape sequence '\p'

C:\Users\Carolina\AppData\Local\Temp\ipykernel_12840\3286184054.py:4: SyntaxWarning:

invalid escape sequence '\p'



['.\\preprocessor_leads.pkl']

In [48]:
joblib.dump(bagging_model, '.\modelo_bagging.pkl')

<>:1: SyntaxWarning:

invalid escape sequence '\m'

<>:1: SyntaxWarning:

invalid escape sequence '\m'

C:\Users\Carolina\AppData\Local\Temp\ipykernel_12840\2946507688.py:1: SyntaxWarning:

invalid escape sequence '\m'



['.\\modelo_bagging.pkl']